In [1]:
import pandas as pd
import plotly.express as px

In [2]:
imonitor = pd.read_csv('data/imonitor_1303.csv')
imonitor.head()

C:\Users\mogam\AppData\Local\Temp\ipykernel_9448\2048010857.py:1: DtypeWarning: Columns (4,11,15,24,25,42,56,62,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  imonitor = pd.read_csv('data/imonitor_1303.csv')


,Survey ID,Data of feedback collection,Facility name and MFL Code if applicable,Facility ownership,Please specify,County,What is your month; and year of birth,How do you consider yourself?,What is the highest level of education you completed?,Please specify.1,...,how long do you wait on average to get a service; which service was that?,Do you consider the waiting time for lab test results long?,how long do you wait on average to get your lab test result?,Does the facility offer support groups?,Specify the support group you belong to,In your opinion are the services offered at this facility youth friendly?,What measures have been put in place to create GBV awareness and its harmful effects within the community?,Please Specify,PWD In your opinion are the services offered at this facility persons-with-disability friendly?,What are the top 1-3 things you don’t like about this facility with regards to care and treatment?
0,2390063,2024-02-28,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1977-09-03,Male,Primary school,NaN,...,NaN,No,NaN,Yes,Adults,Yes,Presence of GBV Desk;,Chiefs office,Yes,NaN
1,2390062,2024-02-28,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1972-08-12,Female,Secondary school,NaN,...,NaN,No,NaN,No,NaN,Yes,Presence of GBV Desk;,Chiefs office,NaN,NaN
2,2390061,2024-02-28,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1984-08-31,Female,Primary school,NaN,...,NaN,Yes,2 hours,No,NaN,Yes,Presence of GBV Desk;,Chiefs office,Yes,NaN
3,2390060,2024-02-28,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1977-05-07,Female,Primary school,NaN,...,NaN,No,NaN,No,NaN,Yes,Presence of GBV Desk;,Police station,NaN,NaN
4,2390059,2024-02-28,BABA DOGO HEALTH CENTRE,GOK,NaN,Nairobi,1987-06-13,Male,Vocational training or technician,NaN,...,1 hour,Yes,2 hours,Yes,Adults,Yes,Presence of GBV Desk;,Police station,NaN,NaN


In [3]:
imonitor.shape

(46549, 85)

In [4]:
# Find and drop columns that contain "Please specify" or "Please Specify"
cols_to_drop = [col for col in imonitor.columns if "Please specify" in col or "Please Specify" in col]

# Drop these columns from the DataFrame in a single operation
imonitor.drop(cols_to_drop, axis=1, inplace=True)

In [5]:
imonitor.shape

(46549, 69)

In [6]:
column_name_mapping = {
    "Survey ID": "SurveyID",
    "Organization name coordinating the feedback from the clients": "OrgFeedbackCoordinator",
    "Data of feedback collection": "FeedbackDate",
    "Facility name and MFL Code if applicable": "FacilityName",
    "Facility ownership": "FacilityOwnership",
    "County": "County",
    "What is your month; and year of birth": "BirthMonthYear",
    "How do you consider yourself? ": "SelfIdentity",
    "What is the highest level of education you completed?": "EducationLevel",
    "What is your current marital status? ": "MaritalStatus",
    "Which county do you currently live in?": "ResidenceCounty",
    "What are your sources of income?": "IncomeSources",
    "For how long have you been accessing services (based on the expected package of services) in this facility?": "ServiceAccessDuration",
    "Are you aware of the package of services that you are entitled to?": "ServicesAwareness",
    "According to you; which HIV related services are you likely to receive in this facility?": "ExpectedHIVServices",
    "Is there a service that you needed that was not provided?": "UnprovidedService",
    "Facility name": "UnprovidedServiceFacilityName",
    "For that service that was not provided; were you referred?": "ReferralForUnprovidedService",
    "If referred; did you receive the service where you were referred to?": "ReferralServiceReceived",
    "If Yes which Service/Test/Medicine ": "ReceivedServiceDetail",
    "On a scale of 1 to 5; how satisfied are you with the package of services received in this facility? If 1 is VERY UNSATISFIED and 5 is VERY SATISFIED.": "ServiceSatisfaction",
    "What did you like about the services you received?": "ServicesLiked",
    "What did you not like about the services you received?": "ServicesDisliked",
    "In your opinion what would you like to be improved?": "ImprovementSuggestions",
    "Do you face any challenges when accessing the services at the facility?": "AccessChallenges",
    "Common issues that can be added in the drop-down box": "CommonIssuesDropdown",
    "In your opinion what can be done to improve access to the services you seek at the facility?": "AccessImprovementSuggestions",
    "Was confidentiality considered while you were being served?": "Confidentiality",
    "Are there age-appropriate health services for specific groups?": "AgeAppropriateServices",
    "Does the facility allow you to share your concerns with the administration?": "ConcernsSharing",
    "Do you know your health-related rights as a client of this facility?": "RightsAwareness",
    "Have you ever been denied services at this facility?": "ServiceDenial",
    "Facility name": "ServiceDenialFacilityName",
    "Why": "ServiceDenialSpecify",
    "Are you comfortable with getting services at this facility": "ComfortWithServices",
    "Have you ever been counseled?": "CounselingReceived",
    "Did you identify any gaps in the facility when you tried to access the services": "IdentifiedGaps",
    "Service type": "ServiceGapsType",
    "Are the HIV testing services readily available when required? ": "HIVTestingAvailability",
    "Have you ever Interrupted your treatment?": "TreatmentInterruption",
    "Are the PMTCT services readily available when required?": "PMTCTServiceAvailability",
    "Were reasons provided as to why these services were not available?": "PMTCTServiceNonavailabilityReasons",
    "Are the HIV prevention; testing; treatment and care services adequate for KPs? ": "KPServiceAdequacy",
    "Were reasons provided as to why these services were not available?.1": "KPServiceNonavailabilityReasons",
    "In your opinion are the services offered at this facility youth friendly?": "YouthFriendlyServices",
    "In your opinion are the services offered at this facility PWD friendly?": "PWDFriendlyServices",
    "What are the barriers to uptake of VMMC by males 25+years and above?": "VMMCBarriers",
    "What are some of the current site level practices that community members like and would love to maintain for KP/PP ?": "KPCommunityPreferredPractices",
    "What would you like this facility to change/do better?": "ChangeSuggestions",
    "What measures have been put in place to create GBV awareness and its harmful effects within the community? ": "GBVAwarenessMeasures",
    "Throughout your visit what did you find interesting/pleasing about this facility that should be emulated by other facilities?": "PositiveObservations",
    "What do you think can be improved": "GeneralImprovementSuggestions",
    "Anything else that you would like to mention?": "AdditionalComments",
    "What are the top 1-3 things you like about this facility with regards to care and treatment? ": "TopFacilityFeatures",
    "Facility Level": "FacilityLevel",
    "Facility Operation times": "OperationTimes",
    "Facility Operation Days ": "OperationDays",
    "What are your preferred days of visiting the facility": "PreferredVisitDays",
    "What are your preferred time of visiting the facility": "PreferredVisitTimes",
    "According to you, which HIV related service/tests/medicine are you likely to receive in this facility?": "ExpectedHIVServices2",
    "Is there a service/test/medicine that you needed that was not provided?": "UnprovidedService2",
    "which service/test/medicine?": "UnprovidedServiceDetail",
    "On a scale of 1-5; how clean do you find the facility?": "FacilityCleanliness",
    "How do you reach this facility?": "FacilityAccessMode",
    "How long does it take to reach this facility?": "FacilityAccessTime",
    "On a scale of 1-5; how accessible do you find this facility?": "FacilityAccessibility",
    "Do you consider the waiting time to be seen at this facility long?": "WaitingTimeOpinion",
    "how long do you wait on average to get a service; which service was that?": "AverageWaitingTime",
    "Do you consider the waiting time for lab test results long?": "LabResultsWaitingTimeOpinion",
    "how long do you wait on average to get your lab test result?": "AverageLabResultsWaitingTime",
    "Does the facility offer support groups?": "SupportGroupAvailability",
    "Specify the support group you belong to": "SpecifySupportGroup",
    "In your opinion are the services offered at this facility youth friendly?.1": "YouthFriendlyServices2",
    "What measures have been put in place to create GBV awareness and its harmful effects within the community? .1": "GBVAwarenessMeasures2",
    "PWD In your opinion are the services offered at this facility persons-with-disability friendly?": "PWDFriendlyServicesOpinion2",
    "What are the top 1-3 things you don’t like about this facility with regards to care and treatment?": "TopFacilityDislikes"
}

df = imonitor.rename(columns=column_name_mapping)

In [7]:
for column in df.columns:
    print(column)

SurveyID
FeedbackDate
FacilityName
FacilityOwnership
County
BirthMonthYear
SelfIdentity
EducationLevel
MaritalStatus
ResidenceCounty
IncomeSources
ServiceAccessDuration
ServicesAwareness
ExpectedHIVServices
UnprovidedService
ServiceDenialFacilityName
ReferralForUnprovidedService
ReferralServiceReceived
ReceivedServiceDetail
ServiceSatisfaction
ServicesLiked
ServicesDisliked
ImprovementSuggestions
AccessChallenges
CommonIssuesDropdown
AccessImprovementSuggestions
Confidentiality
AgeAppropriateServices
ConcernsSharing
RightsAwareness
ServiceDenial
Facility name.1
ServiceDenialSpecify
ComfortWithServices
Have you ever been counseled? 
IdentifiedGaps
ServiceGapsType
HIVTestingAvailability
TreatmentInterruption
PMTCTServiceAvailability
PMTCTServiceNonavailabilityReasons
KPServiceAdequacy
KPServiceNonavailabilityReasons
VMMCBarriers
KPCommunityPreferredPractices
ChangeSuggestions
PositiveObservations
GeneralImprovementSuggestions
AdditionalComments
TopFacilityFeatures
FacilityLevel
Operation

In [8]:
missing_percentage = df.isnull().mean() * 100

# print("Percentage of missing values per column:\n", missing_percentage)

In [9]:
threshold = 100
columns_to_drop = missing_percentage[missing_percentage > threshold].index.tolist()
print("Columns to drop:", columns_to_drop)

Columns to drop: []


In [10]:
df.drop(columns=columns_to_drop, axis=1, inplace=True)
df.shape

(46549, 69)

In [11]:
columns_to_clean1 = [
    'WaitingTimeOpinion',
    'LabResultsWaitingTimeOpinion'
]

def replace_dont_know(df, column):
    df[column] = df[column].replace("Dont Know", "I don't know", regex=False)
    return df

for column in columns_to_clean1:
    df = replace_dont_know(df, column)

In [12]:
columns_to_clean2 = [
    'FacilityCleanliness',
    'FacilityAccessibility'
    ]

def replace_mixed_with_text(df, column_name):
    def replace_value(value):
        satisfaction_map = {
            1: 'Very Unsatisfied',
            2: 'Unsatisfied',
            3: 'Okay',
            4: 'Satisfied',
            5: 'Very Satisfied'
        }
        if isinstance(value, str) and value[0].isdigit():
            num = int(value[0])
        elif isinstance(value, int):
            num = value
        else:
            return value

        return satisfaction_map.get(num, value)

    df[column_name] = df[column_name].apply(replace_value)
    return df

for column in columns_to_clean2:
    df = replace_mixed_with_text(df, column)

In [13]:
def standardize_satisfaction(df, column_name):
    # Mapping for consolidating variations of satisfaction levels
    satisfaction_map = {
        '5': 'Very Satisfied',
        5.0: 'Very Satisfied',
        '4': 'Satisfied',
        4.0: 'Satisfied',
        '3': 'Okay',
        3.0: 'Okay',
        '2': 'Unsatisfied',
        2.0: 'Unsatisfied',
        '1': 'Very Unsatisfied',
        1.0: 'Very Unsatisfied',
        'Dissatisfied': 'Unsatisfied'
    }
    
    # Replace values based on the map
    df[column_name] = df[column_name].replace(satisfaction_map)
    return df

df = standardize_satisfaction(df, 'ServiceSatisfaction')


In [14]:
value_counts = df['ServiceSatisfaction'].value_counts()

# Create a bar chart directly using the value counts Series
fig = px.bar(value_counts, x=value_counts.index, y=value_counts.values, labels={'x': 'Satisfaction Levels', 'y': 'Counts'}, title='Value Counts for Satisfaction with Facility Services')

# Show the figure
fig.show()
